In [1]:
# modify these for your own computer
repo_directory = '/Users/Michael/Documents/GitHub/law-net/'

data_dir = '/Users/Michael/Desktop/network_data/'

import os
import numpy as np
import re
import sys
import matplotlib.pyplot as plt
import glob
import cPickle  as pickle
from collections import OrderedDict


# graph package
import igraph as ig


# stat
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering

import scipy.sparse
import random
import itertools


# our code
sys.path.append(repo_directory + 'code/')
from summarize_clusters import *

sys.path.append(repo_directory + 'vertex_metrics_experiment/code/')
from bag_of_words import * 

# which network to download data for
network_name = 'scotus' # 'federal', 'ca1', etc


# some sub directories that get used
raw_dir = data_dir + 'raw/'
subnet_dir = data_dir + network_name + '/'
text_dir = subnet_dir + 'textfiles/'
nlp_dir = subnet_dir + 'nlp/'
nlp_sub_dir = nlp_dir + 'bow_tfidf/' #tfidf matrix (and other info, i.e. vocab) computed from bag-of-words matrix
nlp_bow_dir = nlp_dir + 'bow/' #bag-of-words matrix (and other info, i.e. vocab)
nlp_df_sub_dir = nlp_dir + 'bow_tfidf_df/'

# csv location
csv_dir = "C:/Users/Michael/Documents/GitHub/law-net/csv/"

# all the file paths for .txt files
file_paths = glob.glob(text_dir + '*.txt')

# all opinions
all_the_opinions = all_opinions(file_paths)

# jupyter notebook settings
%load_ext autoreload
%autoreload 2
%matplotlib inline

## load tf-idf vectors (min_df = 0.3, max_df = 0.7)
**tfidf_matrix** = (row_index, column_index): tf_idf value (**CSR FORMAT**)  
**op_id_to_bow_id** = opinion_id (corresponds to row indices)  
**vocab** = all the words in tfidf_matrix (correspond to column indices)

In [2]:
tfidf_matrix, op_id_to_bow_id, vocab = load_tf_idf(nlp_df_sub_dir)

In [3]:
# convert to dense matrix (easier computation on K-means)
tfidf_matrix = tfidf_matrix.todense()

# Graph Clustering
focus on largest connected component on **undirected scotus**

In [6]:
# load the graph
G = ig.Graph.Read_GraphML(subnet_dir + network_name +'_network.graphml')

# limit ourselves to cases upto and including 2015 since we are missing some textfiles from 2016
G = G.subgraph(G.vs.select(year_le=2015))

# make graph undirected
Gud = G.copy()
Gud = Gud.as_undirected()

# get largest connected componenet
components = Gud.clusters(mode='STRONG')
g = components.subgraphs()[np.argmax(components.sizes())]

# CL ids of cases in largest connected component
CLids = g.vs['name']

## modularity on undirected scotus
"For a given division of the network's vertices into some modules, modularity reflects the concentration of edges within modules compared with random distribution of links between all nodes regardless of modules"--*Wikipedia*

In [7]:
%%time 

# modularity clustering
cd_modularity = g.community_fastgreedy() # .as_clustering().membership

mod_clust = cd_modularity.as_clustering()

print mod_clust.summary()

# save clusters in pandas
graph_clusters = pd.Series(mod_clust.membership, index=g.vs['name'])

Clustering with 24724 elements and 126 clusters
Wall time: 1min 40s


## walktrap on undirected scotus

In [8]:
%%time

# walktrap clustering
cd_walktrap = g.community_walktrap()

wt_clust = cd_walktrap.as_clustering()

print wt_clust.summary()

# save clusters in pandas
walktrap_clusters = pd.Series(wt_clust.membership, index=g.vs['name'])

Clustering with 24724 elements and 2264 clusters
Wall time: 2min 22s


# NLP Clustering
K-Means, Gaussian Mixture Models (GMM), Hierarchical Clustering

## K-Means Clustering on tf-idf (tweaked df)

### K-means clustering with K = 10

In [9]:
%%time

# set number of clusters
num_clusters = 10

# run kmeans
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

nlp_tfidf_clusters = km.labels_.tolist()

Wall time: 35.2 s


### K-means clustering with K = 100

In [10]:
%%time

# set number of clusters
num_clusters = 100

# run kmeans
km2 = KMeans(n_clusters=num_clusters)
km2.fit(tfidf_matrix)

nlp_tfidf_clusters2 = km2.labels_.tolist()

Wall time: 2min 17s


### K-means clustering with K = 1000

In [11]:
%%time

# set number of clusters
num_clusters = 1000

# run kmeans
km3 = KMeans(n_clusters=num_clusters)
km3.fit(tfidf_matrix)

nlp_tfidf_clusters3 = km3.labels_.tolist()

Wall time: 12min 44s


## GMM Clustering on tf-idf (tweaked df)

### GMM Clustering with K = 10

In [12]:
%%time

# set number of components
num_components = 10

# run GMM
gmm = GaussianMixture(n_components=num_components)
gmm.fit(tfidf_matrix)

gmm_clusters = gmm.predict(tfidf_matrix).tolist()
gmm_clusters = map(int, gmm_clusters)

Wall time: 11min 22s


### GMM Clustering with K = 100 

In [13]:
%%time

# set number of components
num_components = 100

# run GMM
gmm2 = GaussianMixture(n_components=num_components)
gmm2.fit(tfidf_matrix)

gmm_clusters2 = gmm2.predict(tfidf_matrix).tolist()
gmm_clusters2 = map(int, gmm_clusters2)

Wall time: 6min 1s


### GMM Clustering with K = 1000

In [14]:
%%time

# set number of components
num_components = 1000

# run GMM
gmm3 = GaussianMixture(n_components=num_components)
gmm3.fit(tfidf_matrix)

gmm_clusters3 = gmm3.predict(tfidf_matrix).tolist()
gmm_clusters3 = map(int, gmm_clusters3)

Wall time: 46min 2s


## Hierarchical Clustering on tf-idf (tweaked df)

### Hierarchical Clustering with K = 10

In [4]:
%%time

# set number of components
num_clusters = 10

# run Hierarchical Clustering (Agglomerative with linkage = 'ward', 'complete', 'average')
'''
1. ward: minimizes the variance of the clusters being merged.
2. average: uses the average of the distances of each observation of the two sets.
3. complete or maximum: uses the maximum distances between all observations of the two sets

note:
manhattan/L1 distance is often good for sparse features, or sparse noise: 
    i.e. many of the features are zero, as in text mining using occurences of rare words.
'''

hc = AgglomerativeClustering(n_clusters = num_clusters, affinity='euclidean', linkage='ward')
hc_clusters = hc.fit_predict(tfidf_matrix).tolist()
hc_clusters = map(int, hc_clusters)

Wall time: 4min 42s


### Hierarchical Clustering with K = 100

In [5]:
%%time

# set number of components
num_clusters = 100

# run Hierarchical Clustering (Agglomerative with linkage = 'ward', 'complete', 'average')
'''
1. ward: minimizes the variance of the clusters being merged.
2. average: uses the average of the distances of each observation of the two sets.
3. complete or maximum: uses the maximum distances between all observations of the two sets

note:
manhattan/L1 distance is often good for sparse features, or sparse noise: 
    i.e. many of the features are zero, as in text mining using occurences of rare words.
'''

hc2 = AgglomerativeClustering(n_clusters = num_clusters, affinity='euclidean', linkage='ward')
hc_clusters2 = hc2.fit_predict(tfidf_matrix).tolist()
hc_clusters2 = map(int, hc_clusters2)

Wall time: 4min 22s


### Hierarchical Clustering with K = 1000

In [6]:
%%time

# set number of components
num_clusters = 1000

# run Hierarchical Clustering (Agglomerative with linkage = 'ward', 'complete', 'average')
'''
1. ward: minimizes the variance of the clusters being merged.
2. average: uses the average of the distances of each observation of the two sets.
3. complete or maximum: uses the maximum distances between all observations of the two sets

note:
manhattan/L1 distance is often good for sparse features, or sparse noise: 
    i.e. many of the features are zero, as in text mining using occurences of rare words.
'''

hc3 = AgglomerativeClustering(n_clusters = num_clusters, affinity='euclidean', linkage='ward')
hc_clusters3 = hc3.fit_predict(tfidf_matrix).tolist()
hc_clusters3 = map(int, hc_clusters3)

Wall time: 4min 21s


## Compare NLP clustering (tfidf) vs graph clustering

In [15]:
#clusters = pd.DataFrame(index=normalized_text_dict.keys(), columns=['nlp', 'graph'])
clusters = pd.DataFrame(index=op_id_to_bow_id, columns=[])

# add in communities 
clusters['mod'] = graph_clusters

# consider nodes not considered in CD to be their own cluster
# i.e. nodes outside the largest connected component
clusters['mod'].fillna(max(graph_clusters) + 1, inplace=True)

# make formatting
clusters['mod'] = clusters['mod'].astype(np.int)

# add in walktrap clusters
clusters['wt'] = walktrap_clusters
clusters['wt'].fillna(max(walktrap_clusters) + 1, inplace=True)
clusters['wt'] = clusters['wt'].astype(np.int)

# add in NLP clusters
clusters['km_10'] = nlp_tfidf_clusters
clusters['km_100'] = nlp_tfidf_clusters2
clusters['km_1000'] = nlp_tfidf_clusters3

clusters['gmm_10'] = gmm_clusters
clusters['gmm_100'] = gmm_clusters2
clusters['gmm_1000'] = gmm_clusters3

clusters['hc_10'] = hc_clusters
clusters['hc_100'] = hc_clusters2
clusters['hc_1000'] = hc_clusters3

clusters.to_csv(csv_dir + "clusters_tweak_df.csv")

### load cluster csv saved in current directory

In [16]:
clusters = pd.read_csv(csv_dir + 'clusters_tweak_df.csv')
clusters

,Unnamed: 0,mod,wt,km_10,km_100,km_1000,gmm_10,gmm_100,gmm_1000,hc_10,hc_100,hc_1000
0,145658,1,5,4,0,927,3,3,966,7,38,43
1,89370,3,294,4,38,413,3,99,247,7,38,143
2,89371,0,35,7,92,129,7,68,646,9,63,680
3,89372,0,3,0,16,271,5,88,491,7,38,483
4,89373,0,3,6,93,722,1,24,369,3,80,530
5,89374,2,4,6,23,149,1,77,75,5,58,454
6,89375,2,5,2,13,988,5,88,793,7,38,581
7,89376,2,6,2,37,136,5,72,995,7,38,344
8,89377,2,7,1,48,196,3,74,262,1,96,198
9,89378,2,7,2,21,632,5,75,444,7,12,642
